# Create Individual Deforestation Files From Hansen Dataset

The purpose of this notebook is two fold:
1. Extract pixels:Takes the 'lossyear' image from the Hansen et al. (2013) dataset and creates a tiff file deforestation_year for each year desired. Take the 'treecover2000' and filter for pixels greater than 10%. 
2. 'lossyear' and 'treecover2000' also need to be cropped so that pixels are only within the boundary of the active property of that year. 

# Import  libraries 

In [1]:
import os
import sys
from pathlib import Path
import rasterio
from rasterio.merge import merge
from rasterio.mask import mask
from rasterio.transform import from_origin
import geopandas as gpd
import pandas as pd
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt


# Import Constants

In [2]:

# Get the current working directory
current_dir = os.path.abspath('')

# Search for the 'constants.py' file starting from the current directory and moving up the hierarchy
project_root = current_dir
while not os.path.isfile(os.path.join(project_root, 'constants.py')):
    project_root = os.path.dirname(project_root)

# Add the project root to the Python path
sys.path.append(project_root)


In [3]:
from constants import HANSEN_LOSSYEAR_FILEPATHS, HANSEN_TREECOVER_FILEPATH, STUDY_BOUNDARY_PATH, LUP_YEAR_PATHS, OUTPUT_PATH

In [4]:
study_boundary_gdf = gpd.read_file(STUDY_BOUNDARY_PATH)

'''rasterio.open() expects a single file path as a string, 
 pass the file path string directly. Since HANSEN_LOSSYEAR_FILEPATHS is a list, 
 you can access the file path string by indexing the list with [0].
 '''
# Reproject the study boundary to match the Hansen raster CRS
with rasterio.open(HANSEN_LOSSYEAR_FILEPATHS[0]) as src:
    hansen_crs = src.crs

# Reproject the study boundary GeoDataFrame to match the Hansen raster CRS    
study_boundary_gdf_reprojected = study_boundary_gdf.to_crs(hansen_crs)
# Get the geometry column from the reprojected GeoDataFrame
study_boundary_geom_reprojected = study_boundary_gdf_reprojected.geometry

In [5]:
with rasterio.open(HANSEN_LOSSYEAR_FILEPATHS[0]) as hansen:
    hansen_array = hansen.read(1)
    hansen_crs = hansen.crs
    out_transform = hansen.transform
    out_meta = hansen.meta

# Extract Tree Cover >= 10%

To extract pixels with 10% and above tree cover and write the resulting raster:

1. Read the tree cover raster data.
2. Create a binary mask for pixels with tree cover equal to or greater than 10%.
3. Write the binary mask to a new raster file.

In [12]:
# Step 1: Read the tree cover raster data
with rasterio.open(HANSEN_TREECOVER_FILEPATH[0]) as src:
    tree_cover_array = src.read(1)
    tree_cover_transform = src.transform
    tree_cover_meta = src.meta

# Step 2: Create a mask for pixels with tree cover equal to or greater than 10%
'''np.where() to create a new array, masked_array,
that contains the original tree cover values 
where the condition (tree cover >= 10%) is met and 0 for the other pixels. '''
tree_cover_threshold = 10
masked_array = np.where(tree_cover_array >= tree_cover_threshold, tree_cover_array, 0)

# Step 3: Write the masked array to a new raster file
output_dir = os.path.join(OUTPUT_PATH[0], 'tree_cover_10_percent_and_above_00')
os.makedirs(output_dir, exist_ok=True)

output_filepath = os.path.join(output_dir, 'tree_cover_10_percent_and_above_00.tif')
with rasterio.open(output_filepath, 'w', **tree_cover_meta) as dst:
    dst.write(masked_array, 1)
    dst.transform = tree_cover_transform

# Extract-Mask Pixels Treecover2010

In [7]:
def extract_pixels_by_year_cumulative(raster_data, start_year, end_year):
    year_data = {}
    cumulative_pixels = np.zeros_like(raster_data, dtype=int)
    
    for year in range(start_year, end_year + 1):
        year_pixels = (raster_data == year).astype(int)
        cumulative_pixels += year_pixels
        year_data[year] = cumulative_pixels.copy()

        # Print unique values for each year
        unique_values = np.unique(year_pixels)
        print(f"Unique values for year {year + 2000}: {unique_values}") # Add 2000 to the year to get the correct year values
    return year_data



In [14]:
pixels_by_year = extract_pixels_by_year_cumulative(hansen_array, 1, 10)

Unique values for year 2001: [0 1]
Unique values for year 2002: [0 1]
Unique values for year 2003: [0 1]
Unique values for year 2004: [0 1]
Unique values for year 2005: [0 1]
Unique values for year 2006: [0 1]
Unique values for year 2007: [0 1]
Unique values for year 2008: [0 1]
Unique values for year 2009: [0 1]
Unique values for year 2010: [0 1]


In [15]:
output_dir = os.path.join(OUTPUT_PATH[0], 'deforestation-cumulative_0110')
os.makedirs(output_dir, exist_ok=True)
# Step 1: Create a cumulative deforestation raster for years 1 to 10
deforestation_1_to_10 = np.zeros_like(hansen_array, dtype=int)

for year in range(1, 11):
    deforestation_1_to_10 += pixels_by_year[year]

# Step 2: Write this to a new raster file
output_filepath = os.path.join(output_dir, 'deforestation_1_to_10.tif')
with rasterio.open(output_filepath, 'w', **out_meta) as dst:
    dst.write(deforestation_1_to_10, 1)
    dst.transform = out_transform

# Step 3: Read the newly created deforestation raster file
with rasterio.open(output_filepath) as src:
    deforestation_1_to_10 = src.read(1)

# Step 4: Mask the tree cover data with the deforestation data
masked_tree_cover_array = np.where(deforestation_1_to_10 > 0, tree_cover_array, 0)

# Step 5: Reclassify the masked tree cover data to have a value of 1 for forest and 0 for non-forest
reclassified_tree_cover_array = np.where(masked_tree_cover_array >= tree_cover_threshold, 1, 0)

# Step 6: Write the reclassified array to a new raster file
output_filepath = os.path.join(output_dir, 'reclassified_tree_cover_10.tif')
with rasterio.open(output_filepath, 'w', **tree_cover_meta) as dst:
    dst.write(reclassified_tree_cover_array, 1)
    dst.transform = tree_cover_transform


# Create Deforestation 11-20

In [16]:
output_dir = os.path.join(OUTPUT_PATH[0], 'deforestation-cumulative_0110')
os.makedirs(output_dir, exist_ok=True)
# Generate cumulative deforestation data for years 12 to 21
pixels_by_year_11_to_20 = extract_pixels_by_year_cumulative(hansen_array, 11, 20)

# Create a cumulative deforestation raster for years 12 to 21
deforestation_11_to_20 = np.zeros_like(hansen_array, dtype=int)

for year in range(11, 21):  # We use 22 because range's upper bound is exclusive
    deforestation_11_to_20 += pixels_by_year_11_to_20[year]

# Write this to a new raster file
output_filepath_11_to_20 = os.path.join(output_dir, 'deforestation_11_to_20.tif')
with rasterio.open(output_filepath_11_to_20, 'w', **out_meta) as dst:
    dst.write(deforestation_11_to_20, 1)
    dst.transform = out_transform


Unique values for year 2011: [0 1]
Unique values for year 2012: [0 1]
Unique values for year 2013: [0 1]
Unique values for year 2014: [0 1]
Unique values for year 2015: [0 1]
Unique values for year 2016: [0 1]
Unique values for year 2017: [0 1]
Unique values for year 2018: [0 1]
Unique values for year 2019: [0 1]
Unique values for year 2020: [0 1]
